In [1]:
import langchain 
print(langchain.__version__)

0.0.248


In [83]:
from dotenv import load_dotenv, find_dotenv
__ = load_dotenv(find_dotenv())#read the local .env file


In [2]:
import os
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
os.environ["OPENAI_API_BASE"] = "https://mazcaeanclabopenai01.openai.azure.com"


In [3]:
# # Set this to `azure`
# export OPENAI_API_TYPE=azure
# # The API version you want to use: set this to `2023-05-15` for the released version.
# export OPENAI_API_VERSION=2023-05-15
# # The base URL for your Azure OpenAI resource. You can find this in the Azure portal under your Azure OpenAI resource.
# export OPENAI_API_BASE=https://your-resource-name.openai.azure.com  
# # The API key for your Azure OpenAI resource.  You can find this in the Azure portal under your Azure OpenAI resource.
# export OPENAI_API_KEY=<your Azure OpenAI API key>

In [3]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter,TokenTextSplitter
from langchain import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import DirectoryLoader
import magic
import os
import nltk

In [4]:
from langchain.document_loaders import PyPDFLoader

loader =  PyPDFLoader("https://www.rogers.com/cms/pdf/en/Consumer_SUG_V20.pdf")
# Load up your text into documents
data = loader.load()

In [5]:
from transformers import AutoTokenizer, AutoModel

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')


In [6]:
#customize splitter which count token length instead of length, control the tokens to be less than 230
#(256 as the upper limit of the embedding model)
def huggingface_tokenizer_length(text: str) -> int:
    return len(tokenizer.encode(text))
text_splitter = RecursiveCharacterTextSplitter(chunk_size=230, chunk_overlap=20,length_function=huggingface_tokenizer_length)
# Split your documents into texts
texts = text_splitter.split_documents(data)

In [7]:
for i in texts[:10]:
    print(len(i.page_content))
print("_____")
for i in texts[:10]:
    print(huggingface_tokenizer_length(i.page_content))

445
595
666
774
586
666
815
731
209
836
_____
116
138
155
166
127
152
164
156
47
190


In [8]:
from langchain.embeddings import OpenAIEmbeddings
embeddings=OpenAIEmbeddings(deployment='text-embedding-ada-002',
                                model='text-embedding-ada-002',
                                chunk_size=8)

In [9]:
# Get your docsearch ready
from langchain.vectorstores import Chroma
docsearch = Chroma.from_documents(texts, embeddings)

In [10]:
docsearch

In [11]:
from langchain.prompts import PromptTemplate
prompt_template = """
        You are a customer service agent of Rogers Communications which is a telecommunication company in Canada.
        You are helpful and honest on answering customers' questions.  

        Use the following information about Rogers Communications to respond to 
        the customer's query acting as a customer service agent.
        Context: {context}                                

        Keep the response brief and only use the information given by the context. If you don't know the answer, just say that you don't know, don't try to make up an answer.

        Customer: {question}
        Customer service agent:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [13]:
from langchain.chat_models import AzureChatOpenAI
llm = AzureChatOpenAI(
    deployment_name="gpt-4-32k",
    model_name="gpt-4-32k"
)

In [14]:
print(llm)

cache=None verbose=False callbacks=None callback_manager=None tags=None metadata=None client=<class 'openai.api_resources.chat_completion.ChatCompletion'> model_name='gpt-4-32k' temperature=0.7 model_kwargs={} openai_api_key='27c9d423022649f1976f8a36029deea4' openai_api_base='https://mazcaeanclabopenai01.openai.azure.com' openai_organization='' openai_proxy='' request_timeout=None max_retries=6 streaming=False n=1 max_tokens=None tiktoken_model_name=None deployment_name='gpt-4-32k' openai_api_type='azure' openai_api_version='2023-05-15'


In [57]:
chain_type_kwargs = {"prompt": PROMPT, "document_separator": "<<<<>>>>>"}
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", 
                                 retriever=docsearch.as_retriever(search_kwargs={"k": 2}), 
                                 chain_type_kwargs=chain_type_kwargs,
                                 return_source_documents=True)

### LLM-Generated examples

In [58]:
examples = [
    {
        "query": "how to download ringtones?",
        "answer": "To download ringtones, please follow these steps:\
            1. Select the Internet icon from your main menu and then select the Shop link from your mobile internet home page or text MUSIC to 555 for a link to the Music and Tones page.\
            2. Check out categories such as Top Sellers and Spotlights by selecting Ringtones.\
            3. Pick a ringtone of your choice, then click on the Download link. Once downloaded, an option will appear to set the new download as your ringtone"
    },
    {
        "query": "How are my services invoiced for the first bill?",
        "answer": "Your first bill includes both partial and regular charges. The regular charges include your monthly plan and any additional fees that are billed in advance for the upcoming month. Partial charges are calculated based on the length of time between your activation date and your invoice date. This covers your usage for that period."
    }
]

In [17]:
from langchain.evaluation.qa import QAGenerateChain
example_gen_chain = QAGenerateChain.from_llm(llm)

In [19]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:10]]
)

C:\Users\yuncheng.liang\AppData\Roaming\Python\Python39\site-packages\langchain\chains\llm.py:303: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [20]:
new_examples[0]

{'qa_pairs': {'query': 'What is the website to get personalized info, How-To videos and support for Rogers services?',
  'answer': 'The website is rogers.com/urstartup.'}}

In [21]:
new_examples[1]

{'qa_pairs': {'query': "How do you add new contacts to your phone's address book?",
  'answer': "To add new contacts, find your phone’s Contacts or Address Book icon on the menu screen. Select 'Add Contact' and fill in the person’s details, then save your entry."}}

In [22]:
new_examples[3]

{'qa_pairs': {'query': 'What are some of the features available once registered with My Rogers?',
  'answer': 'The features available once registered with My Rogers include visiting the Getting Started Centre, accessing your account, paying your bill, and monitoring your usage.'}}

In [50]:
new_examples[0]['qa_pairs']

{'query': 'What is the website to get personalized info, How-To videos and support for Rogers services?',
 'answer': 'The website is rogers.com/urstartup.'}

In [59]:
for i in new_examples:
    examples.append(i['qa_pairs'])

In [60]:
examples

[{'query': 'how to download ringtones?',
  'answer': 'To download ringtones, please follow these steps:            1. Select the Internet icon from your main menu and then select the Shop link from your mobile internet home page or text MUSIC to 555 for a link to the Music and Tones page.            2. Check out categories such as Top Sellers and Spotlights by selecting Ringtones.            3. Pick a ringtone of your choice, then click on the Download link. Once downloaded, an option will appear to set the new download as your ringtone'},
 {'query': 'How are my services invoiced for the first bill?',
  'answer': 'Your first bill includes both partial and regular charges. The regular charges include your monthly plan and any additional fees that are billed in advance for the upcoming month. Partial charges are calculated based on the length of time between your activation date and your invoice date. This covers your usage for that period.'},
 {'query': 'What is the website to get perso

## Manual Evaluation

In [25]:
langchain.debug = True
qa({"query": examples[0]["query"]})

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "how to download ringtones?"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "how to download ringtones?",
  "context": "to the original point of purchase.Mobile Backup\nHARdw ARe RetuRN PoliCy\nGet the most out of each battery charge \nso your phone is ready whenever you are.Tip BAtteRy PeRfoRMANCe  VAlue \nPACksincluded inVAlue \nPACksincluded in\n1  Select the Internet icon \nfrom your main menu. \nThen select the Shop link from your mobile internet home page or text MUSIC  \nto 555 for a link to the \nMusic and Tones page.\n2  Select Ringtones to check \nout categories such as Top Sellers and Spotlights.\n3   Select a ringtone, then click on the Download  \nlink. Once downloaded, an option will appear to set the ne

{'query': 'how to download ringtones?',
 'result': 'To download ringtones, please follow these steps:\n\n1. Select the Internet icon from your main menu. \n2. Then select the Shop link from your mobile internet home page or text MUSIC to 555 for a link to the Music and Tones page.\n3. Select Ringtones to check out categories such as Top Sellers and Spotlights.\n4. Select a ringtone, then click on the Download link. Once downloaded, an option will appear to set the new download as your ringtone.\n\nFor more details and applicable conditions, you can visit rogers.com/mobilebackup.',
 'source_documents': [Document(page_content='to the original point of purchase.Mobile Backup\nHARdw ARe RetuRN PoliCy\nGet the most out of each battery charge \nso your phone is ready whenever you are.Tip BAtteRy PeRfoRMANCe  VAlue \nPACksincluded inVAlue \nPACksincluded in\n1  Select the Internet icon \nfrom your main menu. \nThen select the Shop link from your mobile internet home page or text MUSIC  \nto 5

In [26]:
langchain.debug = False

## LLM assisted evaluation

In [70]:
qa.return_source_documents=False

In [72]:
predictions=qa.apply(examples)

In [73]:
predictions

[{'query': 'how to download ringtones?',
  'answer': 'To download ringtones, please follow these steps:            1. Select the Internet icon from your main menu and then select the Shop link from your mobile internet home page or text MUSIC to 555 for a link to the Music and Tones page.            2. Check out categories such as Top Sellers and Spotlights by selecting Ringtones.            3. Pick a ringtone of your choice, then click on the Download link. Once downloaded, an option will appear to set the new download as your ringtone',
  'result': 'To download ringtones, please follow these steps:\n\n1. Select the Internet icon from your main menu. Then select the Shop link from your mobile internet home page or text MUSIC to 555 for a link to the Music and Tones page.\n2. Select Ringtones to check out categories such as Top Sellers and Spotlights.\n3. Select a ringtone, then click on the Download link. Once downloaded, an option will appear to set the new download as your ringtone.

In [74]:
from langchain.evaluation.qa import QAEvalChain
eval_chain = QAEvalChain.from_llm(llm)

In [75]:
graded_outputs = eval_chain.evaluate(examples, predictions)

In [77]:
graded_outputs

[{'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'}]

In [78]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['results'])
    print()

Example 0:
Question: how to download ringtones?
Real Answer: To download ringtones, please follow these steps:            1. Select the Internet icon from your main menu and then select the Shop link from your mobile internet home page or text MUSIC to 555 for a link to the Music and Tones page.            2. Check out categories such as Top Sellers and Spotlights by selecting Ringtones.            3. Pick a ringtone of your choice, then click on the Download link. Once downloaded, an option will appear to set the new download as your ringtone
Predicted Answer: To download ringtones, please follow these steps:

1. Select the Internet icon from your main menu. Then select the Shop link from your mobile internet home page or text MUSIC to 555 for a link to the Music and Tones page.
2. Select Ringtones to check out categories such as Top Sellers and Spotlights.
3. Select a ringtone, then click on the Download link. Once downloaded, an option will appear to set the new download as your rin